In [1]:
!pip install torch torchvision torchaudio timm albumentations pandas opencv-python scikit-learn numpy tqdm matplotlib torch-xla

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.6/93.6 MB 18.3 MB/s eta 0:00:00


In [2]:
# ====================
# STEP 1: Imports & Environment Setup
# ====================
import os
import random
import timm
import torch
import torch.nn as nn
import torch.optim as optim
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import cv2
from sklearn.model_selection import KFold
import numpy as np
from tqdm import tqdm
from torch.amp import autocast, GradScaler
from sklearn.metrics import mean_absolute_error

# Fixed Albumentations version warning
os.environ['NO_ALBUMENTATIONS_UPDATE'] = '1'

# Set device and seed for reproducibility
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if device.type == 'cuda':
        torch.cuda.manual_seed_all(seed)

set_seed(42)


# Setup base and image directory
BASE_DIR = "/kaggle/input/lacuna-solar-survey-challenge"
IMAGE_DIR = os.path.join(BASE_DIR, "images")
CHECKPOINT_DIR = "/kaggle/working/"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)  # Ensure the directory exists

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:


# ====================
# STEP 2: Dataset with Metadata
# ====================
class SolarPanelDataset(Dataset):
    def __init__(self, dataframe, transform=None, to_train=True):
        self.dataframe = dataframe.reset_index(drop=True)
        self.transform = transform
        self.to_train = to_train
        self.placement_map = {"roof": 0, "openspace": 1, "r_openspace": 2, "S-unknown": 3}

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        # Construct the full image path using IMAGE_DIR
        img_path = row["path"]
        image = cv2.imread(img_path)
        if image is None:
            raise ValueError(f"Image not found at {img_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

        # Prepare metadata vector: first element indicates img_origin, next 4 are one-hot for placement
        metadata = torch.zeros(5, dtype=torch.float32)
        metadata[0] = 1.0 if row["img_origin"] == "D" else 0.0
        placement = self.placement_map.get(row["placement"], 3)
        metadata[1 + placement] = 1.0  # One-hot encoding

        if self.transform:
            image = self.transform(image=image)['image']
        if self.to_train:
            target = torch.tensor([row["boil_nbr"], row["pan_nbr"]], dtype=torch.float32)
            return image, metadata, target
        return image, metadata

# ====================
# STEP 3: Model Definition
# ====================
class EfficientNetV2Meta(nn.Module):
    def __init__(self):
        super(EfficientNetV2Meta, self).__init__()
        self.backbone = timm.create_model("tf_efficientnetv2_b3", pretrained=True, num_classes=0)
        self.meta_processor = nn.Sequential(
            nn.Linear(5, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64)
        )
        self.attention = nn.MultiheadAttention(embed_dim=64, num_heads=4)
        self.regressor = nn.Sequential(
            nn.Linear(self.backbone.num_features + 64, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 2),
            nn.Softplus()  # Suitable for count predictions
        )

    def forward(self, image, metadata):
        img_features = self.backbone(image)
        meta_features = self.meta_processor(metadata.unsqueeze(0))
        attn_output, _ = self.attention(meta_features, meta_features, meta_features)
        combined = torch.cat([img_features, attn_output.squeeze(0)], dim=1)
        return self.regressor(combined)

# ====================
# STEP 4: Data Augmentation
# ====================
IMG_SIZE = (512, 512)
train_transform = A.Compose([
    A.RandomResizedCrop(size=IMG_SIZE, scale=(0.7, 1.0)),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.GaussianBlur(blur_limit=(3, 7), p=0.3),
    A.CLAHE(clip_limit=4.0, p=0.5),
    A.HueSaturationValue(p=0.3),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Resize(height=IMG_SIZE[0], width=IMG_SIZE[1]),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# ====================
# STEP 5: Training & Validation Functions
# ====================
def train_one_fold(train_df, fold, epochs=20, batch_size=16):
    print(f"Starting training for fold {fold+1}")

    # Aggregate train_df by ID and update the image path using IMAGE_DIR
    train_df = train_df.groupby("ID").agg({
        "boil_nbr": "sum",
        "pan_nbr": "sum",
        "img_origin": "first",
        "placement": "first"
    }).reset_index()
    train_df["path"] = train_df["ID"].apply(lambda x: os.path.join(IMAGE_DIR, f"{x}.jpg"))

    # Create K-Fold splits
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    splits = list(kf.split(train_df))
    train_idx, val_idx = splits[fold]

    train_ds = SolarPanelDataset(train_df.iloc[train_idx], transform=train_transform)
    val_ds = SolarPanelDataset(train_df.iloc[val_idx], transform=test_transform)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size * 2, shuffle=False, num_workers=2, pin_memory=True)

    model = EfficientNetV2Meta().to(device)
    criterion = nn.HuberLoss(delta=1.0)
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
    scaler = GradScaler()

    best_mae = float('inf')

    for epoch in range(epochs):
        # Training loop
        model.train()
        train_loss = 0.0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
        for images, meta, targets in pbar:
            images = images.to(device, non_blocking=True)
            meta = meta.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)

            optimizer.zero_grad()
            with autocast(device_type=device.type):
                outputs = model(images, meta)
                loss = criterion(outputs, targets)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()
            pbar.set_postfix(loss=f"{loss.item():.4f}")

        # Validation loop
        model.eval()
        val_loss = 0.0
        preds, truths = [], []
        with torch.no_grad():
            for images, meta, targets in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]"):
                images = images.to(device, non_blocking=True)
                meta = meta.to(device, non_blocking=True)
                targets = targets.to(device, non_blocking=True)

                with autocast(device_type=device.type):
                    outputs = model(images, meta)
                    loss = criterion(outputs, targets)

                val_loss += loss.item()
                preds.append(outputs.cpu().numpy())
                truths.append(targets.cpu().numpy())

        # Compute average loss and MAE
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        preds = np.concatenate(preds)
        truths = np.concatenate(truths)
        mae = mean_absolute_error(truths, preds)

        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val MAE: {mae:.4f}")

        # Save best model
        if mae < best_mae:
            best_mae = mae
            #checkpoint_path = f"best_model_fold{fold}.pth"
            checkpoint_path = os.path.join(CHECKPOINT_DIR, f"best_model_fold{fold}.pth")
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Saved new best model to {checkpoint_path}")

        scheduler.step()

    return best_mae

# ====================
# STEP 6: Inference with TTA
# ====================
def predict(test_df, model_paths, batch_size=32):
    test_df["path"] = test_df["ID"].apply(lambda x: os.path.join(IMAGE_DIR, f"{x}.jpg"))
    test_ds = SolarPanelDataset(test_df, transform=test_transform, to_train=False)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    predictions = np.zeros((len(test_df), 2))
    for path in model_paths:
        model = EfficientNetV2Meta().to(device)
        state_dict = torch.load(path, map_location=device)
        model.load_state_dict(state_dict)
        model.eval()

        tta_preds = []
        with torch.no_grad():
            for images, meta in tqdm(test_loader, desc="Inference"):
                images = images.to(device)
                meta = meta.to(device)
                with autocast(device_type=device.type):
                    outputs = model(images, meta)
                tta_preds.append(outputs.cpu().numpy())

        predictions += np.concatenate(tta_preds)

    return predictions / len(model_paths)

In [4]:

# ====================
# STEP 7: Main Execution
# ====================
if __name__ == "__main__":
    # Load data using the BASE_DIR
    train_df = pd.read_csv(os.path.join(BASE_DIR, "Train.csv"))
    test_df = pd.read_csv(os.path.join(BASE_DIR, "Test.csv"))

    folds = 3
    model_paths = []

    # Train each fold
    for fold in range(folds):
        print(f"Training fold {fold+1}/{folds}")
        best_mae = train_one_fold(train_df, fold=fold, epochs=52, batch_size=32)
        model_paths.append(f"best_model_fold{fold}.pth")

    # Perform inference
    predictions = predict(test_df, model_paths, batch_size=64)

    # Prepare submission files
    submission = pd.DataFrame({
        "ID": np.repeat(test_df["ID"].values, 2),
        "Target": predictions.flatten()
    })
    submission["ID"] += np.where(
        submission.groupby("ID").cumcount() == 0,
        "_boil",
        "_pan"
    )
    submission.to_csv("/kaggle/working/submission_original.csv", index=False)

    int_submission = submission.copy()
    int_submission["Target"] = np.round(int_submission["Target"]).astype(int)
    int_submission.to_csv("/kaggle/working/submission_integer.csv", index=False)

    print("Submissions saved with shapes:", submission.shape, int_submission.shape)

Training fold 1/3
Starting training for fold 1


model.safetensors:   0%|          | 0.00/57.9M [00:00<?, ?B/s]

Epoch 1/52 [Val]: 100%|██████████| 11/11 [00:56<00:00,  5.10s/it]


Epoch 1/52 - Train Loss: 2.1364, Val Loss: 1.6291, Val MAE: 1.9026
Saved new best model to /kaggle/working/best_model_fold0.pth


Epoch 2/52 [Val]: 100%|██████████| 11/11 [00:49<00:00,  4.46s/it]


...

Epoch 51/52 - Train Loss: 0.4210, Val Loss: 1.2912, Val MAE: 1.5390


Epoch 52/52 [Val]: 100%|██████████| 11/11 [00:47<00:00,  4.36s/it]


Epoch 52/52 - Train Loss: 0.4030, Val Loss: 1.2794, Val MAE: 1.5294


<ipython-input-3-a6b2a91e5f84>:193: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device)
Inference: 100%|██████████| 18/18 [01:08

Submissions saved with shapes: (2214, 2) (2214, 2)
